## SCF参数修改

**环境**：
- Python 3.5+
- lxml 3.7.3

**功能**
- 用参考SCF中的某些参数值替换原SCF文件中对应的参数值并生成新的SCF文件

**限制**
- 不支持对同一个节点下多个同名的参数的修改，比如qcitab/pdcp profile/rlc profile下参数

**历史**
- 0.01: 2017/08/07

**TO DO**

In [1]:
# 参数字典
para_dict = {
    # Basic Info
#     'chBw':'20MHz', 'earfcnDL':'1725', 'earfcnUL':'19725', 'earfcn':'38968','tddFrameConf':'2', 'tddSpecSubfConf':'7', 
    'inactivityTimer':'3600', 'maxNrSymPdcch':'3', 
    'actModulationSchemeDl':'64QAM', 'actModulationSchemeUl':'16QAMHighMCS',
    # MIMO & RI
    'dlMimoMode':'Dynamic Open Loop MIMO', 'syncSigTxMode':'TxDiv', 'riEnable':'true', 'riPerM':'1', 'riPerOffset':'-1',
    # Capacity
    'maxNumActUE':'400', 'maxNumActDrb':'1200', 'maxNumRrc':'400', 'maxNumRrcEmergency':'400',
    'maxNumUeDl':'12', 'maxNumUeUl':'12', 'maxNumUeDlDwPTS':'8', 'nCqiDtx':'0',
    'cellSrPeriod':'80ms', 'cqiPerNp':'40ms', 'n1PucchAn':'36', 'nCqiRb':'3', #PUCCH format 2/2b
    'deltaPucchShift':'1','pucchNAnCs':'0', #PUCCH 1/1a/1b
    'nPucchF3Prbs':'0', #PUCCH format 3 
    'prachFreqOff':'4',
    # Feature
    'actDrx':'true','actSmartDrx':'false', 'actAutoPucchAlloc':'true',
    # CA
    'actDLCAggr':'true', 'actULCAggr':'false', 'actInterEnbDLCAggr':'false', 
    'maxNumScells':'2','maxNumCaConfUe':'100', 'maxNumCaConfUeDc':'100', 'maxNumCaConfUe3c':'20',
    # SRS
    'srsActivation':'false','srsBwConf':'0bw','srsOnTwoSymUpPts':'true','srsSubfrConf':'sc4','beamformingType':'nonBeamforming',
    'srsBandwidth':'3hbw','srsUePeriodicity':'10ms',
    # Others
    'recoveryResetDelay':'10000'
}

from lxml import etree
import os

file_path = r'D:\userdata\anliu\Desktop\tmp'
ref_file = r'Golden_SCF.xml-3Cell-example10MHz.xml'
in_file = r'Golden_SCF.xml-3Cell-example20MHz.xml'
out_file = r'Golden_SCF.xml-3Cell-example20MHz_backup.xml'

os.chdir(file_path)

def update_para_dict(p_node, para_dict):
    # 如果p_node下有para_dict中的参数，该标志位置为True
    is_present = False
    # 查找p_node下参数字典中的参数    
    for key, value in para_dict.items():
        # 参数列表中的参数只可能在某个节点下出现一次，所以只需要使用find而不是findall
        node = p_node.find(".//*[@name='%s']" % key)
        # 参数字典中参数可能出现在同一个p_node下，如果p_node下找到参数列表中参数，打印且仅打印一次p_node的attrib
        if node != None and is_present == False:
            is_present = True
            print('='*12 + p_node.attrib['distName'] + '='*12)
        elif node == None:
            continue
        # 如果参数更新，打印参数列表参数值的变化
        if (para_dict[key] == node.text):
            continue
        else:
            node_old_text = para_dict[key]
            para_dict[key] = node.text
            print(key + ': ' + node_old_text + ' -> ' + para_dict[key])      

def replace_para_value(p_node, para_dict):
    # 如果p_node下有para_dict中的参数，该标志位置为True
    is_present = False
    # 查找p_node下参数字典中的参数    
    for key, value in para_dict.items():
        # 参数列表中的参数只可能在某个节点下出现一次，所以只需要使用find而不是findall
        node = p_node.find(".//*[@name='%s']" % key)
        # 参数字典中参数可能出现在同一个p_node下，如果p_node下找到参数列表中参数，打印且仅打印一次p_node的attrib
        if node != None and is_present == False:
            is_present = True
            print('='*12 + p_node.attrib['distName'] + '='*12)
        elif node == None:
            continue
        # 如果参数更新，打印参数列表参数值的变化
        if (node.text == para_dict[key]):
            continue
        else:
            node_old_text = node.text
            node.text = para_dict[key]
            print(key + ': ' + node_old_text + ' -> ' + node.text)  

# 如果参考scf存在，则读取参考scf文件并获得根节点，否则用预先定义的参数字典中的参数值去更新待处理scf中对应的参数值
if ref_file != '':
    ref_tree = etree.parse(ref_file)
    ref_root = ref_tree.getroot()
    # scf文件中需要修改的参数只可能存在于包含属性distName的节点下，所以首先需要找出包含属性distName的节点
    ref_node_list = ref_root.findall(".//*[@distName]")
    # 根据参考scf中参数更新参数字典中的参数值
    # 遍历每一个包含distName属性的节点，替换参数字典中的值。
    print('Start to update parameters dictionary using:', ref_file)
    for p_node in ref_node_list:
        update_para_dict(p_node, para_dict)
    input('Parameters dictionary update complete, press "Enter" key to continue...')
    
# 读取待处理的scf文件并获得根节点  
in_tree = etree.parse(in_file)
in_root = in_tree.getroot()
# scf文件中需要修改的参数只可能存在于包含属性distName的节点下，所以首先需要找出包含属性distName的节点
in_node_list = in_root.findall(".//*[@distName]")
# 遍历每一个包含distName属性的节点，替换参数字典中的值。
print('*'*50)
print('Start to replace parameters of %s with reference scf:'%in_file)
for p_node in in_node_list:
    replace_para_value(p_node, para_dict)
print('*'*50)
print('Parameters update complete!')

# 参数值修改后写入新的scf文件
in_tree.write(out_file, encoding='utf-8', xml_declaration=True) 
print("New scf:", out_file)

Start to update parameters dictionary using: Golden_SCF.xml-3Cell-example10MHz.xml
============MRBTS-1/LNBTS-1============
actDLCAggr: true -> false
recoveryResetDelay: 10000 -> 0
nCqiDtx: 0 -> 100
============MRBTS-1/LNBTS-1/LNCEL-1============
maxNumScells: 2 -> 1
actSmartDrx: false -> true
inactivityTimer: 3600 -> 10
============MRBTS-1/LNBTS-1/LNCEL-2============
============MRBTS-1/LNBTS-1/LNCEL-3============
============MRBTS-1/LNBTS-1/LNCEL-1/LNCEL_FDD-0============
maxNumUeUl: 12 -> 14
maxNumActDrb: 1200 -> 1500
maxNumCaConfUe: 100 -> 50
maxNumCaConfUeDc: 100 -> 50
maxNumActUE: 400 -> 500
maxNumUeDl: 12 -> 14
prachFreqOff: 4 -> 3
============MRBTS-1/LNBTS-1/LNCEL-2/LNCEL_FDD-0============
============MRBTS-1/LNBTS-1/LNCEL-3/LNCEL_FDD-0============
============MRBTS-1/LNBTS-1/LNCEL-1/LNCEL_FDD-0/MPUCCH_FDD-0============
cellSrPeriod: 80ms -> 20ms
maxNumRrcEmergency: 400 -> 480
maxNumRrc: 400 -> 432
n1PucchAn: 36 -> 18
deltaPucchShift: 1 -> 2
============MRBTS-1/LNBTS-1/LNCEL-2/L